In [37]:
# Import Libraries
import requests
import re
import time
import pandas as pd
import numpy as np
import json
import os
import sqlite3
import Levenshtein
from bs4 import BeautifulSoup
from tqdm import tqdm

# Get Genius Keys and Tokens
with open('../../secrets/keys.json', 'r') as file:
    api_data = json.load(file)
    Genius_ID = api_data['GeniusID']
    Genius_Secret = api_data['GeniusSecret']
    Genius_Token = api_data['GeniusToken']

In [14]:
# Read in the list of cleaned artist
with open("../../data/mid/nested_dict_v2.json", "r") as f:
    loaded_dict = json.load(f)
cleaned_names = [artist_info['Cleaned Name'] for artist_info in loaded_dict.values()]
cleaned_names

['The 1975',
 'Biffy Clyro',
 'Alanis Morissette',
 'Burning Spear',
 'Cmat',
 'Supergrass',
 'Neil Young',
 'Raye',
 'John Fogerty',
 'The Script',
 'Brandi Carlile',
 'Kaiser Chiefs',
 'Olivia Rodrigo',
 'Noah Kahan',
 'Nile Rodgers',
 'Rod Stewart',
 'The Libertines',
 'Celeste',
 'The Selecter',
 'Loyle Carner',
 'Busta Rhymes',
 'Gracie Abrams',
 'Franz Ferdinand',
 'Wet Leg',
 'Inhaler',
 'Rizzle Kicks',
 'Fabio & Grooverider',
 'Charli Xcx',
 'Deftones',
 'Ezra Collective',
 'Amyl',
 'Weezer',
 'Beabadoobee',
 'Good Neighbours',
 'Alessi Rose',
 'The Prodigy',
 'Wolf Alice',
 'Snow Patrol',
 'Turnstile',
 'Joy Crookes',
 'Shaboozey',
 'Nadine Shah',
 'Louis Dunford',
 'Maribou State',
 'Badbadnotgood',
 'Denzel Curry',
 'En Vogue',
 'Vieux Farka Toure',
 'Glass Beams',
 'Ca7Riel',
 'Corto.Alto',
 'Doechii',
 'Amaarae',
 'Greentea Peng',
 'Yussef Dayes',
 'Kneecap',
 'Bob Vylan',
 'Nilüfer Yanya',
 'Infinity Song',
 'Overmono',
 'Parcels',
 'The Brian Jonestown Massacre',
 'Goat'

In [15]:
# Attempting one artist pull to see the response
url = "https://api.genius.com/search"
headers = {"Authorization": f"Bearer {Genius_Token}"}
params = {'q': "Amyl"}
    
response = requests.get(url, headers = headers, params = params)
data = response.json()
data

{'meta': {'status': 200},
 'response': {'hits': [{'highlights': [],
    'index': 'song',
    'type': 'song',
    'result': {'annotation_count': 10,
     'api_path': '/songs/2875227',
     'artist_names': 'Amy Shark',
     'full_title': 'Adore by\xa0Amy\xa0Shark',
     'header_image_thumbnail_url': 'https://images.genius.com/cecc4b3dc7a42e780072905927ad4e72.300x300x1.jpg',
     'header_image_url': 'https://images.genius.com/cecc4b3dc7a42e780072905927ad4e72.1000x1000x1.jpg',
     'id': 2875227,
     'lyrics_owner_id': 2034060,
     'lyrics_state': 'complete',
     'path': '/Amy-shark-adore-lyrics',
     'primary_artist_names': 'Amy Shark',
     'pyongs_count': 6,
     'relationships_index_url': 'https://genius.com/Amy-shark-adore-sample',
     'release_date_components': {'year': 2016, 'month': 8, 'day': 1},
     'release_date_for_display': 'August 1, 2016',
     'release_date_with_abbreviated_month_for_display': 'Aug. 1, 2016',
     'song_art_image_thumbnail_url': 'https://images.genius.

In [16]:
Levenshtein.distance("Amyl and the Sniffers", "Amyl")

17

In [17]:
hit_names = [data['response']['hits'][i]['result']['primary_artist']['name'] for i in range(len(data['response']['hits']))]
hit_ids = [data['response']['hits'][i]['result']['primary_artist']['id'] for i in range(len(data['response']['hits']))]
hit_df = pd.DataFrame({"hit_id": hit_ids, "hit_name": hit_names, "artist_name":"Amyl"})
# Use Levenshtein distance to make more accurate
hit_df['lev_dist'] = hit_df.apply(
    lambda row: Levenshtein.distance(row['hit_name'], row['artist_name']),
    axis=1
)
# If the whole of the artist anme is in the hit_name then give a 1
hit_df['in_return'] = hit_df.apply(
    lambda row: 1 if pd.Series(row['hit_name']).str.contains(row['artist_name'], regex=False).any() else 0,
    axis=1
)
hit_df.sort_values(['lev_dist'], inplace = True)
if hit_df['in_return'].min() > 10:
    output_id = np.NaN
elif hit_df['in_return'].sum() > 0:
    ouput_df = hit_df[hit_df['in_return'] == 1]
    output_id = ouput_df.iloc[0, 0]
else:
    output_id = hit_df.iloc[0, 0]

In [21]:
bonk = []
bonk.append("SSas")
bonk[0].lower()

'ssas'

In [47]:
url = "https://api.genius.com/search"
headers = {"Authorization": f"Bearer {Genius_Token}"}
most_likely_name = []
most_likely_id = []
# Do the above but for all artists in the list
for clean_name_i in cleaned_names:
    params = {'q': clean_name_i}
    response = requests.get(url, headers = headers, params = params)
    data_i = response.json()
    # If statement added to save time
    if data_i['response']['hits'][0]['result']['primary_artist']['name'].lower().strip() == clean_name_i.lower().strip():
        output_name = data_i['response']['hits'][0]['result']['primary_artist']['name']
        output_id = data_i['response']['hits'][0]['result']['primary_artist']['id']
    else:
        hit_names = [data_i['response']['hits'][i]['result']['primary_artist']['name'] for i in range(len(data_i['response']['hits']))]
        hit_ids = [data_i['response']['hits'][i]['result']['primary_artist']['id'] for i in range(len(data_i['response']['hits']))]
        hit_df = pd.DataFrame({"hit_id": hit_ids, "hit_name": hit_names, "artist_name":clean_name_i})
        # Use Levenshtein distance to make more accurate
        hit_df['lev_dist'] = hit_df.apply(
            lambda row: Levenshtein.distance(row['hit_name'].lower(), row['artist_name'].lower()),
            axis=1
        )
        # If the whole of the artist anme is in the hit_name then give a 1
        hit_df['in_return'] = hit_df.apply(
            lambda row: 1 if pd.Series(row['hit_name']).str.contains(row['artist_name'], regex=False).any() else 0,
            axis=1
        )
        hit_df.sort_values(['lev_dist'], inplace = True)
        # print(hit_df)
        if hit_df['in_return'].sum() > 0:
            ouput_df = hit_df[hit_df['in_return'] == 1]
            output_id = ouput_df.iloc[0, 0]
            output_name = ouput_df.iloc[0, 1]
        elif hit_df['lev_dist'].min() >= 0.8*len(clean_name_i) or hit_df['in_return'].min() >= 8:
            output_id = np.nan
            output_name = np.nan
        else:
            output_id = hit_df.iloc[0, 0]
            output_name = hit_df.iloc[0, 1]
    most_likely_name.append(output_name)
    most_likely_id.append(output_id)

In [52]:
outputs = pd.DataFrame({"names": cleaned_names, "most_likely_name":most_likely_name, "most_likely_id":most_likely_id})
outputs

,names,most_likely_name,most_likely_id
0,The 1975,The 1975,45824.0
1,Biffy Clyro,Biffy Clyro,62717.0
2,Alanis Morissette,Alanis Morissette,34995.0
3,Burning Spear,Burning Spear,208175.0
4,Cmat,CMAT,2450044.0
...,...,...,...
168,My Baby,NaN,NaN
169,The Horne Section,Alex Horne & The Horne Section,2119663.0
170,Brooke Combe,Brooke Combe,2720203.0
171,Talisk,Talisk,1460995.0


In [ ]:
# With the ids, look at returning the artists url

In [56]:
def get_artist_top_songs(artist_id, access_token, limit=10):
    url = f"https://api.genius.com/artists/{artist_id}/songs"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "sort": "popularity",  # Sort by popularity
        "per_page": limit,
        "page": 1
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        songs = response.json()["response"]["songs"]
        return [(song["title"], song["url"]) for song in songs]
    else:
        print(f"Error: {response.status_code}")
        return []

# Usage
artist_id = "2119663"
access_token = Genius_Token
top_songs = get_artist_top_songs(artist_id, access_token, 10)

for title, url in top_songs:
    print(f"{title}: {url}")

Grandaddy: https://genius.com/Alex-horne-and-the-horne-section-grandaddy-lyrics
My Nan, Your Nan: https://genius.com/Greg-davies-my-nan-your-nan-lyrics
Seasons / Lips: https://genius.com/Alex-horne-and-the-horne-section-seasons-lips-lyrics
Henry Hoover: https://genius.com/Alex-horne-and-the-horne-section-henry-hoover-lyrics
Chris Hoy / Fifty: https://genius.com/Alex-horne-and-the-horne-section-chris-hoy-fifty-lyrics
Concerned / Chinese Five Spice / Thick and Thin: https://genius.com/Alex-horne-and-the-horne-section-concerned-chinese-five-spice-thick-and-thin-lyrics
Less Is More / Thank You: https://genius.com/Alex-horne-and-the-horne-section-less-is-more-thank-you-lyrics
Is It the Police? / Genuinely: https://genius.com/Alex-horne-and-the-horne-section-is-it-the-police-genuinely-lyrics
If I Were / Magic: https://genius.com/Alex-horne-and-the-horne-section-if-i-were-magic-lyrics
Drum and Bass / Music: https://genius.com/Alex-horne-and-the-horne-section-drum-and-bass-music-lyrics
